In [1]:
import re

### TODO: Add recipe 

In [2]:
# Taken from Meal_Planning.ipynb

class Ingredient:
    def __init__(self):
        self.name = name
        self.quantity = quantity
        self.qtype = qtype
    def __add__(self,other):
        assert self.name == other.name, 'Tried and faled to add together two ingredients: ' + self.name + ' ' + other.name
        # Intelligently deal with stuff

def getInfo():
    '''Pulls from grocery info, which is formatted as recipe|ingredient 1,ingredient 2,...
    Output is a list of [recipe name,[[quantity,ingredient name 1],[quantity,ingredient name 2]]]'''
    with open('./_INFO.txt','r') as f:
        info = f.readlines()
        info = [line.split('|') for line in info] # Sometimes I'll leave inspection hints in the _INFO.txt file as well
        
        def sliceIngredients(ingredients):
            ingredients = ingredients.split(',')
            ingredients = [[x[:x.index(' ')],x[x.index(' ')+1:].strip('\n')] for x in ingredients]
            # Had to strip \n because it was keeping it from the end of lines
            return ingredients
        info = [[line[0],sliceIngredients(line[1])] for line in info]
        
    return info

def toIngredients(user_request):
    '''Takes in a list of [recipe name, quantity]
    Looks up ingredients in _INFO.txt, gets ingredients and adds them together.
    If it can\'t find a recipe, it will output that as an error, but will continue adding the rest of the recipes together.'''
    # print('Total meals requested: ' + str(sum([x[1] for x in user_request])))
    info = getInfo() # Get "meal name":[ingredient list] 
    master_meals = [x[0] for x in info]
    ingredients, amounts, errors = [],[],[] # First one is list of strings, second is list of lists (of strings)
    
    # Expand (for example) 5 dan dan noodles to a 5 long list of ['dan dan noodles','dan dan noodles',...]
    # This is easier than trying to multiply the number parts of the ingredients
    new = []
    for meal in user_request:
        new += [[meal[0],1]]*meal[1]
    user_request = new
    
    for meal in user_request: # Check if meals are in the master meal list
        if meal[0] in master_meals: # If they are...
            loc = master_meals.index(meal[0])
            for part in info[loc][1]: # Put the ingredients on the amounts array. (part being ['18oz','ground pork'])
                if part[1] in ingredients:
                    amounts[ingredients.index(part[1])].append(part[0])
                else:
                    ingredients.append(part[1])
                    amounts.append([part[0]])
        else: # If they aren't, add them to the errors list.
            errors.append(meal[0])
    
    # print('Finished looking up ingredients!\nLookup errors: ' + str(len(errors)))
    # if len(errors) > 0:
    #   print(errors)
    
    return ingredients,amounts,errors

# Modified to just throw back recipe names
def checkMenu(supergoods=[]):
    '''Prints all available recipes'''
    info = getInfo()
    return [x[0] for x in info]

In [52]:
prices = {
    'bacon':[[16,'oz'],6.99],
    'egg noodles':[[12,'oz'],1.99],
    'scallions':[[8,''],0.99],
    'eggs':[[18,''],3.99],
    'ginger':[[1,'lb'],2.99],
    'chicken broth':[[4,'cups'],1.99],
    'cornstarch':[[16,'oz'],1.29],
    'hollandaise sauce':[[6,'oz'],2.99],
    'crimini mushrooms':[[1,'lb'],3.99],
    'cremini mushrooms':[[1,'lb'],3.99],
    'peanut butter':[[40,'oz'],4.69],
    'chili sauce':[[28,'oz'],3.99],
    'rice vinegar':[[24,'oz'],5.99],
    'soy sauce':[[20,'oz'],3.99],
    'mirin':[[10,'oz'],4.29],
    'peanut oil':[[24,'oz'],3.99],
    'cellophane noodles':[[6.75,'oz'],2.39],
    'rice stick noodles':[[6.75,'oz'],2.39], # hmmmmmmm
    'shrimp':[[16,'oz'],7.99],
    'honey':[[16,'oz'],5.99],
    'Shaoxing rice wine':[[25,'oz'],11.99],
    'sesame oil':[[12,'oz'],7.99],
    'peas':[[12,'oz'],1.99],
    'hoisin sauce':[[12,'oz'],3.29],
    'bean sprouts':[[12,'oz'],1.49],
    'sweet onion':[[1,'lb'],1.69], # Hopefully it doesn't ask for 1 or 1/2
    'egg roll wrappers':[[50,''],2.49],
    'sweet corn':[[15.25,'oz'],0.79],
    'pineapple':[[1.5,'cups'],0.99],
    'broccoli florets':[[14,'oz'],1.66],
    'green beans':[[14,'oz'],2.99],
    'mango chunks':[[0.5,'lb'],0.99],
    'evaporated milk':[[12,'oz'],0.99],
    'red wine vinegar':[[12,'oz'],2.99],
    'ground turkey':[[16,'oz'],5.99],
    'chicken':[[1,'lb'],2.99],
    'pork shoulder':[[1,'lb'],0.99],
    'butter':[[1,'lb'],3.99],
    'shredded cheese':[[8,'oz'],2.99], # Potential for bulk rates
    'orange juice':[[89,'oz'],5.99],
    'strawberry jam':[[18,'oz'],2.49],
    'fish sauce':[[8.45,'oz'],6.99]
}

conversions = {
    # I could make something that keeps track of both sides of the relationship,
    # but there aren't that many kitchen quantities.
    'cupsoz': 8,
    'tsptbsp': 0.33,
    'lboz': 16,
    'cupstbsp': 16,
    'oztbsp': 2,
    'ozlb': 0.125,
    'tspcups': 0.021,
    'lbtbsp': 15, # Just for ginger
    'lbtsp': 45, # Also just for ginger
    'lbcup': 15/16, # Also just ginger again
    'oztsp': 6,
    'ozcups': 0.125,
    'ozcup':0.125,
    'cupscup':1,
    'cupcups':1
    # 'lbcups': 
    # 'sliceoz': 1 # Specifically for thick cut bacon
}

# This set is for either neglible prices or things I don't usually use
ignore = {'sichuan peppercorns','salt','pepper','roasted peanuts','black pepper','minced garlic', 'garlic cloves',
          'sugar','white pepper','black vinegar','garlic','rice','cashews','raspberries','sea salt','cinnamon',
          'sesame seeds','cinammon','white bread'}
recipes = checkMenu()
removals = {'Hot and Sour Vegetable Soup'}
recipes = [x for x in recipes if x not in removals]


data = [toIngredients([[x,1]]) for x in recipes]
for index, recipe in enumerate(data):
    totalprice = 0
    specialcase = False
    for ingredient, quant in zip(recipe[0],recipe[1]):
        #if ''.join(re.compile('[a-z]+').findall(quant[0])) == '': # If there is no quantity type, add it to the ignore list
        #    # This is a stopgap to just get all the ingredients at once
        #    if ingredient not in ignore:
        #        print('    Ignored ingredient:',ingredient)
        #        ignore.add(ingredient)
        #        specialcase = True
        if ingredient not in ignore:
            try:
                storecost = prices[ingredient]
                modquant = [float(''.join(re.compile('[0-9.]+').findall(quant[0]))), ''.join(re.compile('[a-z]+').findall(quant[0]))]
                qmenu, qpricing = (modquant[1], storecost[0][1])
                if qmenu != qpricing:
                    try:
                        key = storecost[0][1] + modquant[1]
                        if key in {'lbtbsp','lbtsp','lbcup'}:
                            print('    Warning!',key,'used for ingredient',ingredient,'!')
                        cfactor = conversions[key]
                    except KeyError:
                        raise Exception('No conversion exists for',storecost[0][1],'to',modquant[1],'for',ingredient,'with recipe',recipes[index])
                else:
                    cfactor = 1
                relprice = round(modquant[0] / (storecost[0][0] * cfactor) * storecost[1],2)
                totalprice += relprice
            except KeyError:
                raise Exception('Unable to find price in lookup table! I need:',ingredient,'for recipe',recipes[index])
    totalprice = round(totalprice,2)
    if specialcase:
        print(recipes[index],totalprice,'      Ingredients ignored!')
    else:
        print(recipes[index],totalprice)
        #print(ingredient, quant)
# Price lookup table will be something like this:
# [quantityname, amount, price]

    Warning! lbtbsp used for ingredient ginger !
Dan Dan Noodles 11.66
    Warning! lbtsp used for ingredient ginger !
Sweet Chili Shrimp 7.8
Shrimp With Lobster Sauce 11.15
    Warning! lbtsp used for ingredient ginger !
Egg Drop Soup 3.68


Exception: ('No conversion exists for', 'lb', 'to', '', 'for', 'cremini mushrooms', 'with recipe', 'Lighter Egg Foo Young')